ИМПОРТЫ


In [17]:
import time
import requests
import random
import configparser
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


ТЗ
- сделать статистику потраченных денег
- сделать статистику зарегистрированных аккаунтов

ИНИЦИАЛИЗАЦИЯ КОНСТАНТ

In [7]:
# символы для паролей
symbols = 'abcdefjhijklmnopqrstuvwxyz'

# ключик от апи sms_activate
api_key = "9b60b9fed599fb522A4990854A67b18c"
# ссылка апи sms activate
api_url = "https://api.sms-activate.org/stubs/handler_api.php"
# ссылка на вк
vk_url = "https://vk.com/"
# xpath адрес кнопки "sign up"
sign_in = "/html/body/div[10]/div/div/div[2]/div[2]/div[3]/div/div[1]/div[2]/div[2]/div/div/div[1]/button/span"
# xpath адрес поля для ввода номера тела
number_field = "/html/body/div[1]/div/div/div/div/div[1]/div[2]/div/div/div/form/div[1]/section/div[1]/div/div/input"
# xpath адрес кнопки продолжить
continue_button = "/html/body/div[1]/div/div/div/div/div[1]/div[2]/div/div/div/form/div[2]/div[1]/button/span[1]"
# xpath адрес поля для ввода кода
code_field = '//*[@id="otp"]'



ЗАКАЗ НОМЕРА SMS_ACTIVATE



In [4]:
def get_number() -> dict:
    """
    параметры запроса

    действие: получить номер
    страна: канада
    """
    data = {"action" : "getNumber",
                "service" : "vk",
                "api_key" : api_key,
                "country" : 36
                }
    # делаем апи запрос на получение номера
    r = requests.get(api_url, params=data)
    # если успешно
    if "ACCESS_NUMBER" in r.text:
        print(r.text)
        # возвращаем результат True, айди активации, номер 
        return {"result" : True, "id" : r.text.split(":")[1], "number" : r.text.split(":")[2]}
    # если неуспешно
    else:
        print(r.text)
        return {"result" : False, "reason" : r.text}

ПОЛУЧИТЬ СОСТОЯНИЕ АКТИВАЦИИ НОМЕРА

In [24]:
def get_state(activation_id) -> bool:
    """
    параметры запроса

    действие: получить состояние активации
    айди активации: аргумент
    """
    data = {"action" : "getStatus",
                "api_key" : api_key,
                "id" : activation_id
                }
    # делаем запрос на получение состояния активации
    r = requests.get(api_url, params=data)
    # если успешно
    if r.text == "ACCESS_READY":
        print(r.text)
        return True
    # если неуспешно
    else:
        return False
    

ПОЛУЧИТЬ КОД ИЗ СМС

In [1]:
def get_code(activation_id) -> dict:
    """
    параметры запроса

    действие: получить историю активаций
    айди активации: аргумент
    """
    
    data = {"action" : "getHistory",
            "api_key" : api_key}
    # делаем апи запрос на получение истории активаций
    r = requests.get(api_url, params=data)
    print(r)
    # проходимся по каждому элементу в списке
    for element in r:
        # если айди активации наш, то возвращаем смс код
        if activation_id in element:
            return {"result" : True, "code" : element['sms']}
    # иначе возвращаем неудачу
    return {"result" : False, "reason" : "код не найден"}

ГЕНЕРАТОР ПАРОЛЕЙ

In [16]:
def generate_password() -> str:
    password = ''
    for symb in range(10):
        password += random.choice(symbols)
    return password


РАЗРАБОТКА АЛГОРИТМА РЕГИСТРАЦИИ

In [ ]:
# инициализация движка хромиум
browser = webdriver.Chrome()
# вызываем функция получения номера
data = get_number()
# открываем вк
browser.get(vk_url)
# ждем пока страница загрузится (для тестов)
time.sleep(5)
# находим кнопку sign up и нажимаем на нее
browser.find_element(By.XPATH, sign_in).click()
# имитация задержки (для тестов)
time.sleep(5)
# находим поле для ввода номера тела 
field = browser.find_element(By.XPATH, number_field)
# имитация задержки (для тестов)
time.sleep(2)
# очищаем его от мусора
field.clear()
# имитация задержки (для тестов)
time.sleep(2)
# вводим туда наш псевдономер
field.send_keys(data['number'])
# имитация задержки (для тестов)
time.sleep(2)
# находим кнопку продолжить и нажимаем на нее
browser.find_element(By.XPATH, continue_button).click()
# имитация задержки (для тестов)
time.sleep(2)
# проверяем в цикле, пришел ли код на наш номер
while True:
    if get_state(data['id']):
        code = get_code(data['id'])
        break
    # задержка чтобы не перегружать сервак
    time.sleep(3)
# имитация задержки (для тестов)
time.sleep(2)
# находим поле для ввода кода и вводим туда код
browser.find_element(By.XPATH, code_field).send_keys(code)
# имитация задержки (для тестов)
time.sleep(2)




ТЕСТОВЫЕ ВЫЗОВЫ

In [27]:
a = generate_password()
a

'fatmrxvcxn'